In [ ]:
# 案例：美国各州人口密度的统计数据
三个数据集分别放在sate-population\state-areas\state-abbrevs里面

In [2]:
# 用Pandas的read_csv分别读取这三个数据集
import pandas as pd
pop = pd.read_csv(r"D:\Python-Excel\Data-analysis\Python-Data-Science\Objects\data\state-population.csv")
areas = pd.read_csv(r"D:\Python-Excel\Data-analysis\Python-Data-Science\Objects\data\state-areas.csv")
abbrevs = pd.read_csv(r"D:\Python-Excel\Data-analysis\Python-Data-Science\Objects\data\state-abbrevs.csv")

print(pop.head());print(areas.head());print(abbrevs.head())

  state/region     ages  year  population
0           AL  under18  2012   1117489.0
1           AL    total  2012   4817528.0
2           AL  under18  2010   1130966.0
3           AL    total  2010   4785570.0
4           AL  under18  2011   1125763.0
        state  area (sq. mi)
0     Alabama          52423
1      Alaska         656425
2     Arizona         114006
3    Arkansas          53182
4  California         163707
        state abbreviation
0     Alabama           AL
1      Alaska           AK
2     Arizona           AZ
3    Arkansas           AR
4  California           CA


In [ ]:
首先用一个多对一合并获取人口(pop)DataFrame中各州名称缩写对应的全称。我们需要将pop的
state/region列与abbrevs的abbreviation列进行合并，还需要通过how='outer'确保数据没有丢失。

In [3]:
merged = pd.merge(pop, abbrevs, how='outer', 
                  left_on = 'state/region', 
                  right_on = 'abbreviation')

merged = merged.drop('abbreviation', 1)     # 丢弃重复信息
merged.head()

,state/region,ages,year,population,state
0,AL,under18,2012,1117489.0,Alabama
1,AL,total,2012,4817528.0,Alabama
2,AL,under18,2010,1130966.0,Alabama
3,AL,total,2010,4785570.0,Alabama
4,AL,under18,2011,1125763.0,Alabama


In [ ]:
检查是否有缺失值

In [4]:
merged.isnull().any()

state/region    False
ages            False
year            False
population       True
state            True
dtype: bool

In [ ]:
仔细检查是哪些数据缺失了

In [5]:
merged[merged['population'].isnull()].head()

,state/region,ages,year,population,state
2448,PR,under18,1990,NaN,NaN
2449,PR,total,1990,NaN,NaN
2450,PR,total,1991,NaN,NaN
2451,PR,under18,1991,NaN,NaN
2452,PR,total,1993,NaN,NaN


In [ ]:
现在看看是哪些州有缺失

In [6]:
merged.loc[merged['state'].isnull(), 'state/region'].unique()

array(['PR', 'USA'], dtype=object)

In [8]:
merged.loc[merged['state/region'] == 'PR', 'state'] = 'Puerto Rico'
merged.loc[merged['state/region'] == 'USA', 'state'] = 'United States'
merged.isnull().any()

state/region    False
ages            False
year            False
population       True
state           False
dtype: bool

In [9]:
final = pd.merge(merged, areas, on='state', how='left')
final.head()

,state/region,ages,year,population,state,area (sq. mi)
0,AL,under18,2012,1117489.0,Alabama,52423.0
1,AL,total,2012,4817528.0,Alabama,52423.0
2,AL,under18,2010,1130966.0,Alabama,52423.0
3,AL,total,2010,4785570.0,Alabama,52423.0
4,AL,under18,2011,1125763.0,Alabama,52423.0


In [10]:
# 再检查一下数据，看看哪些列还有缺失值没有匹配上
final.isnull().any()

state/region     False
ages             False
year             False
population        True
state            False
area (sq. mi)     True
dtype: bool

In [11]:
# 面积araa列里面还有缺失值，来看看究竟是哪些地区面积缺失
final['state'][final['area (sq. mi)'].isnull()].unique()

array(['United States'], dtype=object)

In [12]:
# 返现面积（areas）DataFrame里面不包含全美国的面积数据，可以插入全国总面积数据
final.dropna(inplace=True)
final.head()

,state/region,ages,year,population,state,area (sq. mi)
0,AL,under18,2012,1117489.0,Alabama,52423.0
1,AL,total,2012,4817528.0,Alabama,52423.0
2,AL,under18,2010,1130966.0,Alabama,52423.0
3,AL,total,2010,4785570.0,Alabama,52423.0
4,AL,under18,2011,1125763.0,Alabama,52423.0


In [13]:
# 我们先选择2010年的各州人口以及总人口数据，然后用query()函数进行快速计算
data2010 = final.query("year == 2010 & ages == 'total'")
data2010.head()

,state/region,ages,year,population,state,area (sq. mi)
3,AL,total,2010,4785570.0,Alabama,52423.0
91,AK,total,2010,713868.0,Alaska,656425.0
101,AZ,total,2010,6408790.0,Arizona,114006.0
189,AR,total,2010,2922280.0,Arkansas,53182.0
197,CA,total,2010,37333601.0,California,163707.0


In [14]:
# 计算人口密度并按序排列，首先对索引进行重置，然后计算出结果：
data2010.set_index('state', inplace=True)
density = data2010['population'] / data2010['area (sq. mi)']

density.sort_values(ascending=False, inplace=True)
density.head()

state
District of Columbia    8898.897059
Puerto Rico             1058.665149
New Jersey              1009.253268
Rhode Island             681.339159
Connecticut              645.600649
dtype: float64

In [15]:
# 查看人口密度最低的几个州的数据
density.tail()

state
South Dakota    10.583512
North Dakota     9.537565
Montana          6.736171
Wyoming          5.768079
Alaska           1.087509
dtype: float64